In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score, confusion_matrix
import operator
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

c:\Users\sahil\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
target = 'redemption_status'

features = ['age_range', 'c_coverage_brand', 'c_coverage_brandt', 'c_coverage_category', 'c_coverage_item', 'c_freq_brand', 'c_freq_brandt', 'c_freq_category', 'c_items_freq_brand', 'c_items_freq_brandt', 'c_items_freq_category', 'c_items_rare_brand', 'c_items_rare_brandt', 'c_items_rare_category', 'c_rare_brand', 'c_rare_brandt', 'c_rare_category', 'c_unique_brand', 'c_unique_brandt', 'c_unique_category', 'c_unique_items', 'campaign_id', 'campaign_type', 'coupon_id', 'customer_id', 'duration', 'family_size', 'income_bracket', 'marital_status', 'no_of_children']

categorical_columns = ['age_range', 'c_freq_brand', 'c_freq_brandt', 'c_freq_category', 'c_rare_brand', 'c_rare_brandt', 'c_rare_category', 'campaign_id', 'campaign_type', 'coupon_id', 'customer_id', 'family_size', 'income_bracket', 'marital_status', 'no_of_children', 'rented', 'overall_freq_brand', 'overall_rare_brand', 'overall_freq_brandt', 'overall_rare_brandt', 'overall_freq_category', 'overall_rare_category']

In [3]:
def preprocess(trainset, testset, categorical_columns, features, target):
    # Combine train and test for uniform preprocessing
    dataset = pd.concat([trainset, testset], ignore_index=True).fillna(0)
    
    # Convert categorical columns to category type
    for column in categorical_columns:
        dataset[column] = dataset[column].astype('category')

    # Filter to selected feature columns
    dataset = dataset[features]

    # One-hot encoding (optional — uncomment if needed)
    # dataset = pd.get_dummies(dataset, drop_first=True)

    # Split back to train and test
    train_len = len(trainset)
    train_features = dataset.iloc[:train_len, :].copy()
    test_features = dataset.iloc[train_len:, :].copy()

    # Add target column back to trainset
    train_processed = pd.concat([trainset[[target]].reset_index(drop=True), train_features.reset_index(drop=True)], axis=1)
    test_processed = test_features.reset_index(drop=True)

    return train_processed, test_processed


In [5]:
trainset = pd.read_csv(r'train1.csv', index_col='id', parse_dates=['start_date','end_date'])
trainset  = pd.DataFrame(trainset)
trainset

,redemption_status,campaign_id,coupon_id,customer_id,campaign_type,start_date,end_date,duration,age_range,marital_status,...,overall_coverage_item,overall_coverage_brand,overall_coverage_brandt,overall_coverage_category,overall_podiscount,overall_pcdiscount,overall_ptdiscount,overall_podiscount_pq,overall_pcdiscount_pq,overall_ptdiscount_pq
id,,,,,,,,,,,,,,,,,,,,,
1,0,13,27,1053,0,2013-05-19,2013-07-05,47,46-55,0.0,...,0.002808,0.015195,1.0,0.421053,-0.219529,-0.001901,-0.221430,-0.226799,-0.002547,-0.229346
2,0,13,116,48,0,2013-05-19,2013-07-05,47,36-45,1.0,...,0.003294,0.018452,1.0,0.631579,-0.134105,-0.015566,-0.149671,-0.134672,-0.014163,-0.148835
6,0,9,635,205,1,2013-03-11,2013-04-12,32,46-55,1.0,...,0.007196,0.033647,1.0,0.578947,-0.172274,-0.021414,-0.193688,-0.149134,-0.019910,-0.169044
7,0,13,644,1050,0,2013-05-19,2013-07-05,47,NaN,NaN,...,0.002916,0.013205,1.0,0.421053,-0.204061,-0.009207,-0.213268,-0.171436,-0.002500,-0.173936
9,0,8,1017,1489,0,2013-02-16,2013-04-05,48,46-55,1.0,...,0.004415,0.019175,1.0,0.421053,-0.205019,-0.004710,-0.209729,-0.202754,-0.006626,-0.209380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128587,0,8,71,1523,0,2013-02-16,2013-04-05,48,70+,1.0,...,0.004807,0.032381,1.0,0.578947,-0.085659,0.000000,-0.085659,-0.080501,0.000000,-0.080501
128589,0,30,547,937,0,2012-11-19,2013-01-04,46,70+,1.0,...,0.003011,0.017366,1.0,0.473684,-0.217481,-0.004803,-0.222285,-0.202415,-0.005312,-0.207727
128590,0,8,754,1004,0,2013-02-16,2013-04-05,48,NaN,NaN,...,0.016148,0.057344,1.0,0.736842,-0.158425,-0.001542,-0.159967,-0.156793,-0.000941,-0.157734


In [8]:
testset = pd.read_csv(r'test1.csv', index_col='id', parse_dates=['start_date','end_date'])
testset  = pd.DataFrame(testset)
testset.head()

,campaign_id,coupon_id,customer_id,campaign_type,start_date,end_date,duration,age_range,marital_status,rented,...,overall_coverage_item,overall_coverage_brand,overall_coverage_brandt,overall_coverage_category,overall_podiscount,overall_pcdiscount,overall_ptdiscount,overall_podiscount_pq,overall_pcdiscount_pq,overall_ptdiscount_pq
id,,,,,,,,,,,,,,,,,,,,,
3,22,869,967,0,2013-09-16,2013-10-18,32,36-45,0.0,0.0,...,0.008884,0.038531,1.0,0.684211,-0.116013,-0.015037,-0.131051,-0.112131,-0.015335,-0.127466
4,20,389,1566,1,2013-09-07,2013-11-16,70,26-35,1.0,0.0,...,0.016391,0.053003,1.0,0.789474,-0.123949,-0.005637,-0.129586,-0.129240,-0.005853,-0.135093
5,22,981,510,0,2013-09-16,2013-10-18,32,26-35,0.0,0.0,...,0.013758,0.037988,1.0,0.578947,-0.322410,-0.019489,-0.341900,-0.332639,-0.019500,-0.352139
8,25,1069,361,1,2013-10-21,2013-11-22,32,18-25,0.0,0.0,...,0.004361,0.021346,1.0,0.421053,-0.185940,0.000000,-0.185940,-0.179631,0.000000,-0.179631
10,17,498,811,1,2013-07-29,2013-08-30,32,NaN,NaN,NaN,...,0.009721,0.037808,1.0,0.578947,-0.196402,-0.000910,-0.197312,-0.184152,-0.000647,-0.184799


In [9]:
trainset, testset= preprocess(trainset, testset,categorical_columns,features,target)

print("Trainset size: {}".format(trainset.shape))
print("Testset size: {}".format(testset.shape))

Trainset size: (78369, 31)
Testset size: (50226, 30)


In [10]:
class Ensemble():
    def __init__(self, models):
        self.__models = models
    
    def predict(self, data):
        pred_sum = np.zeros(len(data))
        for model in self.__models:
            pred_sum = pred_sum + model.predict(data, num_iteration=model.best_iteration)
        return pred_sum / len(self.__models)

In [11]:
def train(X_train, y_train, num_leaves, verbose):
    params = {
        "objective" : "binary",
        "metric" : "auc",
        "boosting": "gbdt",
        "verbosity" : -1,
        "seed": 41,
        "learning_rate": 0.08,
        "num_leaves": num_leaves,
        "pos_bagging_fraction": 1.0,
        "neg_bagging_fraction": 0.09,
        "bagging_freq": 50,
        "feature_fraction": 0.4,
        "early_stopping_round": 100
    }
    
    lgtrain = lgb.Dataset(X_train, label=y_train)

    evals_result = {}
    model = lgb.train(params, lgtrain,num_boost_round = 3000, verbose_eval=verbose, evals_result=evals_result)
    score = evals_result['valid_0']['auc'][-1]  if 'training' in evals_result else None
    print("num_leaves {}: {}".format(num_leaves, score))
    
    return  model
    
def run_lgbm(features, verbose=False):
    X_train, y_train = trainset[features], trainset[target]
    
    
    models = [train(X_train, y_train, num_leaves, verbose) for num_leaves in [15, 20, 25]]
    ensemble_model = Ensemble(models)
    
   
    score= roc_auc_score(y_train, ensemble_model.predict(X_train))
    
    return ensemble_model, score

Feature Verification